# Sales Data Analysis for 2019

## Project Description

This project involves the analysis of sales data collected by a client for the year 2019. The goal is to uncover insights that can help the client optimize their sales strategy, increase revenue, and identify operational inefficiencies. The analysis will follow the CRISP-DM framework, answering key business questions such as:

1. How much money did we make this year? 

2. Can we identify any seasonality in the sales? 

3. What are our best and worst-selling products? 

4. How do sales compare to previous months or weeks? 

5. Which cities are our products delivered to most? 

6. How do product categories compare in revenue generated and quantities ordered? 

7. You are required to show additional details from your findings in your data. 


### Importation of libraries

In [24]:
# Data Handling
import pyodbc
from dotenv import dotenv_values
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime  
from sqlalchemy import create_engine
import os
# Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px

# Statistical Analysis
from statsmodels.tsa.stattools import adfuller
from scipy.stats import ttest_ind
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error


# Feature Processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

# Modelling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import RandomizedSearchCV
import pickle


# Other Packages
import warnings

warnings.filterwarnings("ignore")

# Data Loading


In [25]:
# Directory path where CSV files for January to June are located
directory_path = r'C:\Users\HP\OneDrive\Desktop\capstone project\Sales-Data-Analysis-for-2019\notebook\data'


# List of file names for each month
file_names = [
    "Sales_January_2019.csv",
    "Sales_February_2019.csv",
    "Sales_March_2019.csv",
    "Sales_April_2019.csv",
    "Sales_May_2019.csv",
    "Sales_June_2019.csv"
]

# Generate file paths for each month
file_paths = [os.path.join(directory_path, file_name) for file_name in file_names]

# Load CSV files into separate DataFrames
data_frames = [pd.read_csv(file) for file in file_paths]

# Combine DataFrames into a single DataFrame
first_half_data = pd.concat(data_frames, ignore_index=True)

# View first half of the dataset
first_half_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


#### Loading the second data from SQL database

In [26]:
# Loading environment variables from .env file
environment_variables = dotenv_values('.env')
 
# Getting the values for the credentials set in the .env file
server = environment_variables.get("SERVER")
database = environment_variables.get("DATABASE")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")
 
# Create a connection string
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"
# Connecting to the server
try:
    connection = pyodbc.connect(connection_string)
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful


In [27]:
# Define SQL queries
sql_queries = [
    'SELECT * FROM dbo.Sales_August_2019',
    'SELECT * FROM dbo.Sales_July_2019',
    'SELECT * FROM dbo.Sales_September_2019',
    'SELECT * FROM dbo.Sales_October_2019',
    'SELECT * FROM dbo.Sales_November_2019',
    'SELECT * FROM dbo.Sales_December_2019'
]

# Execute queries and fetch data into DataFrames
data_frames = []
for sql_query in sql_queries:
    df = pd.read_sql(sql_query, connection)
    data_frames.append(df)

# Combine DataFrames into a single DataFrame
second_half_data = pd.concat(data_frames, ignore_index=True)

# View second half of the dataset
second_half_data.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,236670.0,Wired Headphones,2.0,11.990000,2031-08-19 22:21:00.0000000,"359 Spruce St, Seattle, WA 98101"
1,236671.0,Bose SoundSport Headphones,1.0,99.989998,2015-08-19 15:11:00.0000000,"492 Ridge St, Dallas, TX 75001"
2,236672.0,iPhone,1.0,700.000000,2006-08-19 14:40:00.0000000,"149 7th St, Portland, OR 97035"
3,236673.0,AA Batteries (4-pack),2.0,3.840000,2029-08-19 20:59:00.0000000,"631 2nd St, Los Angeles, CA 90001"
4,236674.0,AA Batteries (4-pack),2.0,3.840000,2015-08-19 19:53:00.0000000,"736 14th St, New York City, NY 10001"


### 3. Data Preparation
3.1. Data Cleaning

3.1.1. Standardize columns for both datasets.

In [28]:
# Display the columns for the first half of the dataframe
first_half_data.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')

In [29]:
# Display the columns for the second half of the dataframe
second_half_data.columns

Index(['Order_ID', 'Product', 'Quantity_Ordered', 'Price_Each', 'Order_Date',
       'Purchase_Address'],
      dtype='object')

- The first half seems to have column names like 'Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date','Purchase Address, while the second half has column names like 'Order_ID', 'Product', 'Quantity_Ordered', 'Price_Each', 'Order_Date','Purchase_Address.
- we need to standardize the column names first.

In [30]:
# Rename columns of the first half of the dataset
first_half_data.columns = ['Order_ID', 'Product', 'Quantity_Ordered', 'Price_Each', 'Order_Date', 'Purchase_Address']
first_half_data.columns

Index(['Order_ID', 'Product', 'Quantity_Ordered', 'Price_Each', 'Order_Date',
       'Purchase_Address'],
      dtype='object')

### 3.1.2. Concatenate Dataset

In [31]:
# Concatenate the two halves along the rows axis
full_year_data = pd.concat([first_half_data, second_half_data], ignore_index=True)

# Display the combined dataset
full_year_data.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,141234,iPhone,1,700,01/22/19 21:25,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301"


### 3.1.3. Check data structure

In [32]:
full_year_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order_ID          186110 non-null  object
 1   Product           186305 non-null  object
 2   Quantity_Ordered  186110 non-null  object
 3   Price_Each        186110 non-null  object
 4   Order_Date        186110 non-null  object
 5   Purchase_Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


- From the table above we can observe the number of values in each column and the datatype as well.
- The datatype of Order ID, Quantity Ordered, Price Each, and Order Date are wrong and must be changed to their correct form.
- Other features will be generated and others will be dropped to ensure an accurate dataset for the purpose of this analysis.

In [33]:
f'There are {full_year_data.shape[0]} rows and {full_year_data.shape[1]} columns in the dataset.'

'There are 186850 rows and 6 columns in the dataset.'

### 3.1.4. Handle missing values

In [34]:
# Check null values in the dataset
full_year_data.isnull().sum()

Order_ID            740
Product             545
Quantity_Ordered    740
Price_Each          740
Order_Date          740
Purchase_Address    545
dtype: int64

In [35]:
# Confirm rows with null values in the dataset
missing_values = first_half_data[full_year_data.columns].isnull().any(axis=1)
rows_with_missing_values = first_half_data[missing_values]
rows_with_missing_values.head(50)

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
664,NaN,NaN,NaN,NaN,NaN,NaN
678,NaN,NaN,NaN,NaN,NaN,NaN
797,NaN,NaN,NaN,NaN,NaN,NaN
876,NaN,NaN,NaN,NaN,NaN,NaN
1299,NaN,NaN,NaN,NaN,NaN,NaN
1491,NaN,NaN,NaN,NaN,NaN,NaN
1699,NaN,NaN,NaN,NaN,NaN,NaN
3047,NaN,NaN,NaN,NaN,NaN,NaN
3082,NaN,NaN,NaN,NaN,NaN,NaN
3584,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# View the last 20 rows for the missing values
rows_with_missing_values.tail(20)

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
78221,NaN,NaN,NaN,NaN,NaN,NaN
79484,NaN,NaN,NaN,NaN,NaN,NaN
80030,NaN,NaN,NaN,NaN,NaN,NaN
80054,NaN,NaN,NaN,NaN,NaN,NaN
80134,NaN,NaN,NaN,NaN,NaN,NaN
80373,NaN,NaN,NaN,NaN,NaN,NaN
80999,NaN,NaN,NaN,NaN,NaN,NaN
81356,NaN,NaN,NaN,NaN,NaN,NaN
82758,NaN,NaN,NaN,NaN,NaN,NaN
82979,NaN,NaN,NaN,NaN,NaN,NaN


- Since the rows with missing values in the dataset have all columns to be missing, we cannot impute the missing values hence we will drop the null values in the dataset.

In [37]:
# Drop null values
full_year_data.dropna(inplace=True)

# Check null values to confirm dropping of the null values
full_year_data.isnull().sum()

Order_ID            0
Product             0
Quantity_Ordered    0
Price_Each          0
Order_Date          0
Purchase_Address    0
dtype: int64

### 3.1.5 Remove duplicates

In [38]:
# Check duplicates in the dataset
f'There are {full_year_data.duplicated().sum()} duplicated rows in the dataset.'

'There are 423 duplicated rows in the dataset.'

In [39]:
# Confirm duplicated rows in the dataset
duplicates = full_year_data[full_year_data.duplicated()]
duplicates

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
875,142071,AA Batteries (4-pack),1,3.84,01/17/19 23:02,"131 2nd St, Boston, MA 02215"
1102,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1194,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1897,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2463,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
...,...,...,...,...,...,...
181627,314675.0,AA Batteries (4-pack),1.0,3.84,2026-12-19 09:01:00.0000000,"927 13th St, San Francisco, CA 94016"
182185,315204.0,Wired Headphones,1.0,11.99,2012-12-19 12:41:00.0000000,"680 6th St, San Francisco, CA 94016"
182973,315955.0,ThinkPad Laptop,1.0,999.98999,2026-12-19 17:28:00.0000000,"588 Chestnut St, Seattle, WA 98101"
183200,316173.0,AAA Batteries (4-pack),1.0,2.99,2022-12-19 22:44:00.0000000,"907 Sunset St, Portland, OR 97035"


The duplicated rows in the dataset will be dealt with by keeping the first occurence and dropping every other instance.

In [40]:
# Drop duplicates
full_year_data.drop_duplicates(keep='first', inplace=True)

# Confirm drop
f'There are {full_year_data.duplicated().sum()} duplicated rows in the dataset now.'

'There are 0 duplicated rows in the dataset now.'

### 3.1.6. Check columns unique values

In [41]:
# Check unique values for product column
full_year_data['Product'].unique()

array(['iPhone', 'Lightning Charging Cable', 'Wired Headphones',
       '27in FHD Monitor', 'AAA Batteries (4-pack)',
       '27in 4K Gaming Monitor', 'USB-C Charging Cable',
       'Bose SoundSport Headphones', 'Apple Airpods Headphones',
       'Macbook Pro Laptop', 'Flatscreen TV', 'Vareebadd Phone',
       'AA Batteries (4-pack)', 'Google Phone', '20in Monitor',
       '34in Ultrawide Monitor', 'ThinkPad Laptop', 'LG Dryer',
       'LG Washing Machine', 'Product'], dtype=object)

- The Product value in the Product column must be inspected to validate the value

In [42]:
# Check rows with Product value as value
full_year_data[full_year_data['Product'] == 'Product']

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
1073,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


This row in the dataset must be dropped since it is an invalid row with invalid values.

In [43]:
# Drop row with invalid column values
invalid_values = full_year_data[full_year_data['Product'] == 'Product'].index
full_year_data.drop(index=invalid_values, inplace=True)

In [44]:
# Confirm drop of row
full_year_data[full_year_data['Product'] == 'Product']

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address


In [45]:
# Inspect the Purchase Address column
full_year_data['Purchase_Address']

0                944 Walnut St, Boston, MA 02215
1               185 Maple St, Portland, OR 97035
2          538 Adams St, San Francisco, CA 94016
3             738 10th St, Los Angeles, CA 90001
4                  387 10th St, Austin, TX 73301
                           ...                  
186845    14 Madison St, San Francisco, CA 94016
186846      549 Willow St, Los Angeles, CA 90001
186847          273 Wilson St, Seattle, WA 98101
186848            778 River St, Dallas, TX 75001
186849    747 Chestnut St, Los Angeles, CA 90001
Name: Purchase_Address, Length: 185686, dtype: object

- The purchase address column contains multiple information which we will split for better analysis of the dataset.

- Columns such as Street Name, City, State, and ZIP Code will be extracted from this column after which the column itself will be dropped.

In [46]:
# Check unique values in the Order Date column
full_year_data['Order_Date'].unique()

array(['01/22/19 21:25', '01/28/19 14:15', '01/17/19 13:33', ...,
       '2001-12-19 12:01:00.0000000', '2009-12-19 06:43:00.0000000',
       '2003-12-19 10:39:00.0000000'], dtype=object)

From the output for the unique values in the order date column we observe that there are dates other then 2019 which we will have to get rid of to help us focus our analysis on the date needed alone.

### 3.1.7. Convert data types

In [47]:
# Convert 'Order_Date' to datetime

full_year_data['Order_Date'] = pd.to_datetime(full_year_data['Order_Date'])

# Convert 'Quantity_Ordered' and 'Price_Each' to float


# Convert 'Price Each' column to numeric
full_year_data['Price_Each'] = pd.to_numeric(full_year_data['Price_Each'])

# Convert 'Quantity Ordered' column to numeric
full_year_data['Quantity_Ordered'] = pd.to_numeric(full_year_data['Quantity_Ordered'], downcast='integer')

# Convert 'Order_ID' column in the first dataset to float64
full_year_data['Order_ID'] = pd.to_numeric(full_year_data['Order_ID'], downcast='integer')

In [48]:
full_year_data['Order_Date'].unique()

<DatetimeArray>
['2019-01-22 21:25:00', '2019-01-28 14:15:00', '2019-01-17 13:33:00',
 '2019-01-05 20:33:00', '2019-01-25 11:59:00', '2019-01-29 20:22:00',
 '2019-01-26 12:16:00', '2019-01-05 12:04:00', '2019-01-01 10:30:00',
 '2019-01-22 21:20:00',
 ...
 '2003-12-19 05:50:00', '2023-12-19 14:08:00', '2013-12-19 07:25:00',
 '2011-12-19 21:24:00', '2007-12-19 08:25:00', '2030-12-19 01:06:00',
 '2011-12-19 20:58:00', '2001-12-19 12:01:00', '2009-12-19 06:43:00',
 '2003-12-19 10:39:00']
Length: 142395, dtype: datetime64[ns]

In [49]:
# Check data info to confirm changes
full_year_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185686 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order_ID          185686 non-null  int32         
 1   Product           185686 non-null  object        
 2   Quantity_Ordered  185686 non-null  int8          
 3   Price_Each        185686 non-null  float64       
 4   Order_Date        185686 non-null  datetime64[ns]
 5   Purchase_Address  185686 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int8(1), object(2)
memory usage: 8.0+ MB


### 3.2. Data Transformation
#### 3.2.1. Generate new features

In [50]:
# Filter out 2019 data from the entire dataset provided
data_2019 = full_year_data[full_year_data['Order_Date'].dt.year == 2019]
data_2019.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215"
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035"
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016"
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001"
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301"


In [51]:
# Generate column for sales
data_2019['Sales'] = data_2019['Quantity_Ordered'] * data_2019['Price_Each']

In [52]:
# Split the value in the Purchase Address into Street Address, City, and State/ZIP Code
data_2019[['Street_Address', 'City','State/ZIP_Code']] = data_2019['Purchase_Address'].astype('str').str.split(',', expand=True)
data_2019.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Purchase_Address,Sales,Street_Address,City,State/ZIP_Code
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,"944 Walnut St, Boston, MA 02215",700.00,944 Walnut St,Boston,MA 02215
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,"185 Maple St, Portland, OR 97035",14.95,185 Maple St,Portland,OR 97035
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,"538 Adams St, San Francisco, CA 94016",23.98,538 Adams St,San Francisco,CA 94016
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,"738 10th St, Los Angeles, CA 90001",149.99,738 10th St,Los Angeles,CA 90001
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,"387 10th St, Austin, TX 73301",11.99,387 10th St,Austin,TX 73301


In [53]:
# Split the State/ZIP_Code column into their respective individual columns
data_2019[['State', 'ZIP_Code']] = data_2019['State/ZIP_Code'].astype('str').str.split(expand=True)

In [54]:
# Drop Purchase Address and State/ZIP_Code columns
data_2019.drop(['Purchase_Address', 'State/ZIP_Code'],axis=1, inplace=True)

data_2019.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Sales,Street_Address,City,State,ZIP_Code
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,700.00,944 Walnut St,Boston,MA,02215
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,14.95,185 Maple St,Portland,OR,97035
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,23.98,538 Adams St,San Francisco,CA,94016
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,149.99,738 10th St,Los Angeles,CA,90001
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,11.99,387 10th St,Austin,TX,73301


In [55]:
data_2019['Day_of_Week']=data_2019['Order_Date'].dt.day_name()

In [56]:
# Group product into high-level and basic-level based on their prices

group_list = [] # Create an empty list

for value in data_2019['Price_Each'].values:   # Loop throught the values in the column
    if value > 99.99:
        group_list.append('High Level') # Assign High level if price is greater than 99.99
    else:
        group_list.append('Basic Level')    # Assign Basic level if price is less than 99.99

data_2019['Product_Group'] = group_list # Add the list to the dataframe

In [57]:
# Inspect dataframe
data_2019.head()

,Order_ID,Product,Quantity_Ordered,Price_Each,Order_Date,Sales,Street_Address,City,State,ZIP_Code,Day_of_Week,Product_Group
0,141234,iPhone,1,700.00,2019-01-22 21:25:00,700.00,944 Walnut St,Boston,MA,02215,Tuesday,High Level
1,141235,Lightning Charging Cable,1,14.95,2019-01-28 14:15:00,14.95,185 Maple St,Portland,OR,97035,Monday,Basic Level
2,141236,Wired Headphones,2,11.99,2019-01-17 13:33:00,23.98,538 Adams St,San Francisco,CA,94016,Thursday,Basic Level
3,141237,27in FHD Monitor,1,149.99,2019-01-05 20:33:00,149.99,738 10th St,Los Angeles,CA,90001,Saturday,High Level
4,141238,Wired Headphones,1,11.99,2019-01-25 11:59:00,11.99,387 10th St,Austin,TX,73301,Friday,Basic Level
